## Import Libraries

In [2]:
import os
import json
import cv2
import numpy as np
import base64

## Set Folder Path and File Names

In [ ]:
# Set folder path (adjust if needed)
folder_path = '/home/suhkth/Desktop/DL-Breast-MRI-Tumor-Segmentation/convertData2PNG/labeled_mri_gray'

# File names
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
print(f"Found {len(json_files)} JSON files to process.")

for json_file in json_files:
    # Construct full paths for input JSON and output files
    json_filename = os.path.join(folder_path, json_file)
    slice_name = os.path.splitext(json_file)[0]  # Remove .json extension
    output_image_filename = os.path.join(folder_path, f"{slice_name}_label.png")
    output_text_filename = os.path.join(folder_path, f"{slice_name}_label.txt")

    print(f"Processing: {json_file}")

## Read JSON File and Determine Image Size

In [ ]:
# Load the JSON annotation file
with open(json_filename, 'r') as f:
    data = json.load(f)

# Determine image dimensions
    height, width = (300, 300)  # Default dimensions
    if "imageData" in data and data["imageData"]:
        # Use embedded base64 imageData
        imageData = data["imageData"]
        img_bytes = base64.b64decode(imageData)
        img_arr = np.frombuffer(img_bytes, np.uint8)
        decoded_img = cv2.imdecode(img_arr, cv2.IMREAD_COLOR)
        if decoded_img is not None:
            height, width = decoded_img.shape[:2]
    elif "imagePath" in data and data["imagePath"]:
        # Use imagePath to load the image file
        image_file = os.path.join(folder_path, data["imagePath"])
        if os.path.exists(image_file):
            loaded_img = cv2.imread(image_file)
            if loaded_img is not None:
                height, width = loaded_img.shape[:2]

    print(f"Image dimensions: {height} x {width}")


Determined image dimensions: 300 x 300


## Create a Blank RGB Image

In [7]:
# Create a blank (all-black) image with three color channels (RGB)
blank_img = np.zeros((height, width, 3), dtype=np.uint8)

## Draw the Annotated “tumor” Polygons in Red

In [8]:
# Loop through each shape in the JSON
for shape in data.get('shapes', []):
    # Filter by label "tumor" (case-insensitive)
    if shape.get("label", "").lower() != "tumor":
        continue

    # Get the list of points and convert to NumPy array; round and convert to integers
    pts = np.array(shape.get('points', []), dtype=np.float32)
    pts = np.round(pts).astype(np.int32)
    pts = pts.reshape((-1, 1, 2))
    
    # Fill the polygon on the blank image with red.
    # Note: OpenCV uses BGR, so red is (0, 0, 255).
    cv2.fillPoly(blank_img, [pts], (0, 0, 255))

print("Annotated polygons have been drawn on the image.")


Annotated polygons have been drawn on the image.


## Save the Output Image


In [9]:
# Save the image with the drawn annotations
cv2.imwrite(output_image_filename, blank_img)
print(f"Output image saved to: {output_image_filename}")


Output image saved to: /home/suhkth/Desktop/DL-Breast-MRI-Tumor-Segmentation/convertData2PNG/labeled_mri_gray/slice_0500_post_label.png


## Write the Class Names to a Text File

In [10]:
# Write the two class names to a text file
with open(output_text_filename, 'w') as txt_file:
    txt_file.write("background\ntumor")

print(f"Class names saved to: {output_text_filename}")


Class names saved to: /home/suhkth/Desktop/DL-Breast-MRI-Tumor-Segmentation/convertData2PNG/labeled_mri_gray/slice_0500_post_label.txt
